In [ ]:
import pandas as pd
import glob
import subprocess as sp
import datetime as dt
import numpy as np
import matplotlib.pylab as plt
import matplotlib.dates as mdates

%matplotlib inline

In [ ]:
dirlogs='./'

In [ ]:
# make monitoring file
def make_monitor_file():
    ''' create a csv file with content from monitor package that
    pandas can read '''
    filenames = glob.glob(dirlogs + 'STDOUT.0000.*')
    sp.call('rm monitor.log',shell=True)
    for filename in filenames:
        sp.call("cat " + filename + " | grep %MON | awk '{print $4,$6}' >> monitor.log",shell=True)

make_monitor_file()

In [ ]:
# read monitor.log file into pandas
monitoring = pd.read_csv('monitor.log',sep=' ',names=['gcmvariable','gcmvalue'])

In [ ]:
# create the time axis
# start date can be found in STDOUT looking for startdate
timesteps = monitoring[monitoring.gcmvariable=='time_tsnumber'].gcmvalue.values
seconds_from = monitoring[monitoring.gcmvariable=='time_secondsf'].gcmvalue.values

dates= []
for kt in np.arange(len(seconds_from)):
    dates.append(dt.datetime(2010,1,1,0,0) + dt.timedelta(seconds=seconds_from[kt]))

In [ ]:
# print list of available diagnostics
list_available_diags = set(monitoring.gcmvariable.values)
print list_available_diags

In [ ]:
# monitoring plot
def plot_statistics_of(cvar):
    ''' plots min/max/mean timeseries,
    takes variable without suffix as string (e.g. dynstat_sst)'''
    meanvar = monitoring[monitoring.gcmvariable==cvar+'_mean'].gcmvalue.values
    minvar  = monitoring[monitoring.gcmvariable==cvar+'_min' ].gcmvalue.values
    maxvar  = monitoring[monitoring.gcmvariable==cvar+'_max' ].gcmvalue.values
    
    fig = plt.figure(figsize=[12,8])

    plt.subplot(121)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator())
    plt.fill_between(dates,minvar,maxvar,color='grey')
    plt.plot(dates,meanvar,'r.')
    plt.title('range and mean '+cvar)
    plt.gcf().autofmt_xdate()

    plt.subplot(122)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
    plt.gca().xaxis.set_minor_locator(mdates.DayLocator())
    plt.plot(dates,meanvar,'r.')
    plt.title('zoom on mean '+cvar)
    plt.gcf().autofmt_xdate()
    plt.show()
    
    

In [ ]:
# surface T/S statistics
plot_statistics_of('dynstat_sst')
plot_statistics_of('dynstat_sss')

In [ ]:
# Volume T/S
plot_statistics_of('dynstat_theta')
plot_statistics_of('dynstat_salt')

In [ ]:
# Atmospheric forcing statistics
plot_statistics_of('forcing_qnet')
plot_statistics_of('forcing_empmr')